<a href="https://colab.research.google.com/github/vaguiar/camelyon_dl_2019/blob/data-exploration/training_at_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip  install tf-nightly-2.0-preview

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os

# The code in this notebook should work identically in TF v1 and v2
import tensorflow as tf
import zipfile

import tensorflow.keras.applications as applications
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("You have version", tf.__version__)
assert tf.__version__ >= "2.0" # TensorFlow ≥ 2.0 required

You have version 2.0.0-dev20190511


### Global constants

In [0]:
# Images will be resized to(TARGET_SHAPE, TARGET_SHAPE) as they're read off disk.
TARGET_SHAPE = 150 
BATCH_SIZE = 25

CLASS_MODE = 'categorical'

NUM_OF_CLASSES = 2
CLASS_NAMES = ['tumor', 'non_tumor']

#################
EPOCHS = 50

### Setting up GCP

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
GCP_PROJECT_ID = 'triple-voyage-239123'
GCP_BUCKET_NAME = 'vaa2114_dl_2019'

In [6]:
!gcloud config set project {GCP_PROJECT_ID}

Updated property [core/project].


### Read Data from GCP

In [0]:
LEVEL = 5

In [0]:
TRAIN_DIR = "/tmp/train/"
GCP_TRAIN_DIR = "train/level{}/".format(LEVEL)
LEVEL_TRAIN_DIR = "/tmp/train/level{}".format(LEVEL)

TEST_DIR = "/tmp/test/"
GCP_TEST_DIR = "test/level{}/".format(LEVEL)
LEVEL_TEST_DIR = "/tmp/test/level{}".format(LEVEL)

!rm -rf '$TRAIN_DIR'
!rm -rf '$TEST_DIR'

!mkdir -p '$TRAIN_DIR'
!mkdir -p '$TEST_DIR'

### Download Level data into Train/Test Directories

In [13]:
!gsutil -m cp -r gs://{GCP_BUCKET_NAME}/{GCP_TRAIN_DIR} {TRAIN_DIR}
  
print("\nDownloaded Train Data")

print("# Non Tumor files")
!ls "$LEVEL_TRAIN_DIR/non_tumor/" | wc -l
print("# Tumor files")
!ls "$LEVEL_TRAIN_DIR/tumor/" | wc -l

Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_217600_tumor_035.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_108800_tumor_078.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_217600_tumor_057.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_0_96000_tumor_094.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_19200_tumor_094.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_19200_tumor_101.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_25600_tumor_096.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_25600_tumor_094.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_12800_tumor_094.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_25600_tumor_101.tif...
Copying gs://vaa2114_dl_2019/train/level5/non_tumor/level_5_102400_32000_tumor_096.tif...
Copying gs://vaa2114_dl_201

In [14]:
!gsutil -m cp -r gs://{GCP_BUCKET_NAME}/{GCP_TEST_DIR} {TEST_DIR}
  
print("\nDownloaded Test Data")

print("# Non Tumor files")
!ls "$LEVEL_TEST_DIR/non_tumor/" | wc -l
print("# Tumor files")
!ls "$LEVEL_TEST_DIR/tumor/" | wc -l

Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_0_121600_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_0_0_tumor_059.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_0_0_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_0_128000_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_0_134400_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_12800_0_tumor_059.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_0_217600_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_12800_0_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_12800_12800_tumor_001.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_12800_12800_tumor_059.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_12800_19200_tumor_084.tif...
Copying gs://vaa2114_dl_2019/test/level5/non_tumor/level_5_12800_256

### Util functions

In [0]:
def getModel(conv_base):
  model = Sequential()
  model.add(conv_base)
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(216, activation='relu'))
  model.add(Dense(NUM_OF_CLASSES, activation='softmax'))
  
  return model

In [0]:
# Compile and build

def trainModel(model, total_train, total_val, 
               train_data_gen, val_data_gen, EPOCHS):
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])

  history = model.fit_generator(
        train_data_gen,
        steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
        epochs=EPOCHS,
        validation_data=val_data_gen,
        validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))),
        verbose=2)
  
  return history